In [1]:
# Load libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import numpy as np

In [2]:
# load dataset
train_data = pd.read_csv("train.csv")
# test_data = pd.read_csv("test.csv")
# test_data = test_data.drop("index",axis=1)
# data = pd.concat([train_data,test_data],sort=False)

In [ ]:
data.head()

In [4]:
import gensim

In [5]:
model = gensim.models.word2vec.Word2Vec.load("word2vec.model")

In [ ]:
model["good"]

In [ ]:
len(data.columns)

In [ ]:
cols = list(data.columns)

In [ ]:
cols.remove("price")

In [ ]:
cols.remove("id")

In [ ]:
cols

In [ ]:
len(data["country"].unique())

In [ ]:
pd.get_dummies(data["country"])

In [ ]:
d = pd.get_dummies(train_data[["country","region_1","region_2","province","taster_name","variety"]])
d["points"]= train_data["points"]
d["id"] = train_data['id']

In [18]:
# countries = list(train_data["country"].values)
col_dic = dict()
for column in ['country','province','region_1','region_2','taster_name','variety']:
    col_dic[column] = list(set(list(train_data[column].values)))

num_data = pd.DataFrame() 
for column in col_dic.keys():
    num_data[column] = train_data[column].apply(lambda column_item:col_dic[column].index(column_item))
# num_data["price"] = train_data["price"]
num_data["points"] = train_data["points"]
num_data["id"] = train_data["id"]
num_data.head()

,country,province,region_1,region_2,taster_name,variety,points,id
0,2,450,0,0,0,77,88.870874,32027
1,8,73,179,0,0,265,88.041695,71079
2,8,73,681,0,0,265,94.085021,32440
3,19,85,420,10,0,529,89.869797,124405
4,19,36,81,15,16,549,89.017651,33649


In [ ]:
X_train,y_train,X_test, y_test = d[:175000], data["price"][:175000], d[175000:], data["price"][175000:]

In [19]:
X_train, X_test, y_train, y_test = train_test_split(num_data, train_data["price"], test_size=0.3, random_state=1) # 70% training and 30% test

In [20]:
X_train = X_train.drop(["id"],axis=1)

In [21]:
%%time
# Create Decision Tree classifer object
clf = LinearRegression(n_jobs=3)
# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

CPU times: user 23.2 ms, sys: 0 ns, total: 23.2 ms
Wall time: 27.1 ms


In [22]:
#Predict the response for test dataset
ids = X_test["id"]
X_test = X_test.drop(["id"],axis=1)

In [23]:
y_pred = clf.predict(X_test)

In [24]:
# removing outliers
y_pred[y_pred>2500] = 50
y_pred[y_pred<0]=1

In [25]:
print(y_test[:10],y_pred[:10])

10709      9.0
27315     28.0
129593    38.0
23186     17.0
61957     18.0
128582    22.0
83535     20.0
65190     13.0
100308    28.0
24450     16.0
Name: price, dtype: float64 [21.13378172 46.32216038 47.22774555 58.95720067 11.85374397 33.78060395
 26.68835291 16.61724989 16.28566156  1.        ]


In [ ]:
result = pd.DataFrame()
result['id'] = ids
result["price"] = y_pred

In [ ]:
result["price"]=result["price"].apply(int)

In [ ]:
result.to_csv('group_6_output.csv',index=False)

In [26]:
y_pred = np.int64(y_pred)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.022323809523809524


In [27]:
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE: 34.55780333627372


In [ ]:
model